<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/haystack2x-demos/haystack_rag_services_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This notebook showcases a novel application of Qdrant vector DB in the upcoming Haystack OpenAPI service-based Retriever-Augmented Generation (RAG) system. Given a user query, we retrieve the corresponding OpenAPI specification from Qdrant vector DB. The retrieved service specification is then seamlessly injected into the Haystack pipeline, enabling the dynamic invocation of selected API services. This innovative approach, integrating any OpenAPI spec service with LLMs, opens a new era for RAG systems to generate contextually rich responses from any openapi service.

## 1. Setup
This notebook demos Haystack 2.x service based RAG, using two openapi services: GitHub's compare_branches and SerperDev search API.

Let's install necessary libraries and import key modules to build the foundation for the subsequent steps.

In [1]:
!pip uninstall -y llmx

Found existing installation: llmx 0.0.15a0
Uninstalling llmx-0.0.15a0:
  Successfully uninstalled llmx-0.0.15a0


In [2]:
!pip install -q "grpcio-tools==1.42" sentence-transformers openapi3 jsonref qdrant-haystack haystack-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.3/206.3 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.9 MB/s eta 0:00:00


In [3]:
import getpass
import os
import json
import requests
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
from haystack_integrations.components.retrievers.qdrant import QdrantEmbeddingRetriever

from haystack import Pipeline
from haystack.components.generators.utils import print_streaming_chunk
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.converters import OpenAPIServiceToFunctions
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.connectors import OpenAPIServiceConnector
from haystack.dataclasses import ChatMessage

## 2. Setting up Qdrant vector DB, create indexing pipeline

In [4]:
document_store = QdrantDocumentStore(
    path="./qdrant_v1",
    index="Document",
    embedding_dim=768,
    recreate_index=True
  )

In [5]:
indexing_pipeline = Pipeline()
indexing_pipeline.add_component("spec_to_functions", OpenAPIServiceToFunctions())
indexing_pipeline.add_component("embedder", SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-mpnet-base-v2"))
indexing_pipeline.add_component("writer", DocumentWriter(document_store=document_store))
indexing_pipeline.connect(sender="spec_to_functions", receiver="embedder")
indexing_pipeline.connect(sender="embedder", receiver="writer")

## 3. Index two openapi specs (Github and SerperDev), along with system prompts

In [7]:
indexing_pipeline.run(data={"sources":["https://bit.ly/github_compare", "https://bit.ly/serper_dev_spec"],
                            "system_messages":[requests.get("https://bit.ly/auto_pr_writer_system_prompt").text,
                                               requests.get("https://bit.ly/serper_dev_system_prompt").text]})

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100it [00:00, 3545.99it/s]           


{'writer': {'documents_written': 2}}

In [8]:
retrieval_pipeline = Pipeline()
retrieval_pipeline.add_component("embedder", SentenceTransformersTextEmbedder(model="sentence-transformers/all-mpnet-base-v2"))
retrieval_pipeline.add_component("retriever", QdrantEmbeddingRetriever(document_store=document_store))
retrieval_pipeline.connect(sender="embedder", receiver="retriever")

## 4. API keys, set up simple authentication mechanism

In [9]:
llm_api_key = getpass.getpass("Enter LLM provider api key:")


Enter LLM provider api key:··········


In [10]:
serper_dev_key = getpass.getpass("Enter serperdev api key:")
github_token = getpass.getpass("Enter your github token:")

services_auth = {"SerperDev":serper_dev_key, "Github API": github_token}

Enter serperdev api key:··········
Enter your github token:··········


## 5. Retrieval step - service invocation

Based on the prompt, retrieve the right service spec from Qdrant, inject spec and system prompt into the pipeline

In [11]:
from haystack.utils import Secret

invoke_service_pipe = Pipeline()
invoke_service_pipe.add_component("functions_llm", OpenAIChatGenerator(api_key=Secret.from_token(llm_api_key), model="gpt-3.5-turbo-0613"))
invoke_service_pipe.add_component("openapi_container", OpenAPIServiceConnector(services_auth))
invoke_service_pipe.connect("functions_llm.replies", "openapi_container.messages")

In [17]:
user_prompt = "Search web and tell me why was Sam Altman ousted from OpenAI"
#user_prompt = "Compare branches main and test/benchmarks2.0, in project deepset-ai, repo haystack"

In [18]:
top_k = 1
top_k_result = retrieval_pipeline.run(data={"text": user_prompt, "top_k": top_k})
top_k_docs = top_k_result["retriever"]["documents"][:top_k]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
top_1_document = top_k_docs[0]
openai_functions_definition = json.loads(top_1_document.content)
openapi_spec = top_1_document.meta["spec"]

tools_param = [{"type": "function", "function": openai_functions_definition}]
tool_choice = {"type": "function", "function": {"name": openai_functions_definition["name"]}}

service_response = invoke_service_pipe.run(data={"messages":[ChatMessage.from_user(user_prompt)],
                                                 "generation_kwargs": {"tools": tools_param,
                                                                       "tool_choice": tool_choice},
                                                 "service_openapi_spec": openapi_spec})

## 6. Generate LLM response

Inject service response into LLM context, pair it with system prompt

In [20]:
gen_pipe = Pipeline()
llm = OpenAIChatGenerator(api_key=Secret.from_token(llm_api_key), model="gpt-4-1106-preview", streaming_callback=print_streaming_chunk)
gen_pipe.add_component("llm", llm)

github_pr_prompt_messages = [ChatMessage.from_system(top_1_document.meta["system_message"])] + service_response["openapi_container"]["service_response"]
final_result = gen_pipe.run(data={"messages": github_pr_prompt_messages})

Sam Altman was ousted from his position as CEO of OpenAI on November 17, 2023, by the company's board of directors. The board announced his departure as effective immediately. This decision came after the board lost confidence in his leadership, as detailed on the Wikipedia page dedicated to his removal. However, it appears that after a series of discussions concerning the future of the company, Sam Altman returned to his position as CEO of OpenAI just days after he was ousted.

In [21]:
from IPython.display import display, Markdown
display(Markdown(final_result["llm"]["replies"][0].content))


Sam Altman was ousted from his position as CEO of OpenAI on November 17, 2023, by the company's board of directors. The board announced his departure as effective immediately. This decision came after the board lost confidence in his leadership, as detailed on the Wikipedia page dedicated to his removal. However, it appears that after a series of discussions concerning the future of the company, Sam Altman returned to his position as CEO of OpenAI just days after he was ousted.

## Thank you, questions?

<a href="www.qr-code-generator.com/" border="0" style="cursor:default" rel="nofollow"><img src="https://chart.googleapis.com/chart?cht=qr&chl=https%3A%2F%2Fgithub.com%2Fvblagoje%2Fnotebooks%2Fblob%2Fmain%2Fhaystack2x-demos%2Fhaystack_rag_services_demo.ipynb&chs=180x180&choe=UTF-8&chld=L|2"></a>

## Links:
- https://github.com/deepset-ai/haystack/
- https://haystack.deepset.ai/community
- https://haystack.deepset.ai/advent-of-haystack
- https://x.com/vladblagoje